In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_gene_mut_count_mat, get_multi_exp_max_freq_mut_df, get_mut_type_avg_frac_across_class_df
from util.mut import is_coding_mut, get_original_nuc_mut_range
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_feature_hit_set, is_overlap, get_promoter_range_from_RegulonDB_df_row, NON_K12_EXP_L 

In [2]:
pd.options.display.max_columns = 100

In [3]:
all_muts_df = pd.read_pickle("./data/1_df.pkl")
display(all_muts_df.shape)

(1278, 13)

In [4]:
# Renaming the "Gene" column to "mutation target annotation" because this is more accurate.
all_muts_df = all_muts_df.rename(index=str, columns={"Gene": "mutation target annotation"})

# Build genetic feature annotation
Need to build genetic feature annotations before genomic features annotations since the genetic annotations are used to fill gaps with unknown genomic targets.

## K12 specific genetic annotation

In [5]:
gene_df = pd.read_csv(
    "./data/RegulonDB10/gene.txt", sep="\t", comment='#', header=None)
gene_df.columns = [
    "GENE_ID",
    "GENE_NAME",
    "GENE_POSLEFT",
    "GENE_POSRIGHT",
    "GENE_STRAND",
    "GENE_SEQUENCE",
    "GC_CONTENT",
    "CRI_SCORE",
    "GENE_NOTE",
    "GENE_INTERNAL_COMMENT",
    "KEY_ID_ORG",
    "GENE_TYPE"
]

def get_gene_range(row):
    r = ()
    if not pd.isna(row["GENE_POSLEFT"]) and not pd.isna(row["GENE_POSRIGHT"]):
        r = (int(row["GENE_POSLEFT"]), int(row["GENE_POSRIGHT"])) 
    return r

gene_df["range"] = gene_df.apply(lambda r: get_gene_range(r), axis=1)
gene_df.head()

,GENE_ID,GENE_NAME,GENE_POSLEFT,GENE_POSRIGHT,GENE_STRAND,GENE_SEQUENCE,GC_CONTENT,CRI_SCORE,GENE_NOTE,GENE_INTERNAL_COMMENT,KEY_ID_ORG,GENE_TYPE,range
0,ECK120000001,alr,4265782.0,4266861.0,forward,ATGCAAGCGGCAACTGTTGTGATTAACCGCCGCGCTCTGCGACACA...,55.93,NaN,NaN,NaN,ECK12,NaN,"(4265782, 4266861)"
1,ECK120000002,modB,795862.0,796551.0,forward,ATGATACTGACCGATCCAGAATGGCAGGCAGTTTTATTAAGCCTGA...,54.06,NaN,NaN,NaN,ECK12,NaN,"(795862, 796551)"
2,ECK120000003,cysZ,2531463.0,2532224.0,forward,ATGGTTTCATCATTCACATCTGCCCCACGCAGCGGTTTTTACTATT...,50.13,NaN,NaN,NaN,ECK12,NaN,"(2531463, 2532224)"
3,ECK120000004,dfp,3812731.0,3813951.0,forward,ATGAGCCTGGCCGGTAAAAAAATCGTTCTCGGCGTTAGCGGCGGTA...,53.64,NaN,NaN,NaN,ECK12,NaN,"(3812731, 3813951)"
4,ECK120000005,dcuB,4347404.0,4348744.0,reverse,ATGTTATTTACTATCCAACTTATCATAATACTGATATGTCTGTTTT...,52.27,NaN,NaN,NaN,ECK12,NaN,"(4347404, 4348744)"


In [6]:
all_muts_df["range"] = all_muts_df.apply(get_original_nuc_mut_range, axis=1)
all_muts_df.head()

,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range
2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)"
6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)"
7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)"
8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)"
0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)"


In [7]:
all_muts_df["gene RegulonDB ID"] = all_muts_df.apply(
    lambda r: get_feature_hit_set(
        r["range"], gene_df, "range", "GENE_ID") if r.exp not in NON_K12_EXP_L else set(), axis=1
)  # Maybe instead just have a list of K12 experiments

all_muts_df.head()

,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID
2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556}
6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854}
7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886}
8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431}
0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625}


In [8]:
temp_gene_df = gene_df.copy()
temp_gene_df = temp_gene_df.sort_values(by=["GENE_POSLEFT"])  # Getting the in order according to position
temp_gene_df = temp_gene_df.reset_index()

intergenic_df = pd.DataFrame(columns=["name", "RegulonDB ID", "range"])
# Don't want to iterate to the last one since the gene will be considered in i-1
for i in range(0, len(temp_gene_df) - 1):
    # check if there even exists an intergenic region between genes.
    if (temp_gene_df.loc[i + 1]["GENE_POSLEFT"] - temp_gene_df.loc[i]["GENE_POSRIGHT"]) > 1:
        intergenic_df = intergenic_df.append(
            {"name": str(temp_gene_df.loc[i]["GENE_NAME"]) + '/' + str(temp_gene_df.loc[i + 1]["GENE_NAME"]),
            "RegulonDB ID": temp_gene_df.loc[i]["GENE_ID"] + '/' + temp_gene_df.loc[i + 1]["GENE_ID"],
            "range": (int(temp_gene_df.loc[i]["GENE_POSRIGHT"] + 1), int(temp_gene_df.loc[i + 1]["GENE_POSLEFT"] - 1))},
            ignore_index=True
    )
display(intergenic_df.head())

,name,RegulonDB ID,range
0,thrL/thrA,ECK120001251/ECK120000987,"(256, 336)"
1,thrA/thrB,ECK120000987/ECK120000988,"(2800, 2800)"
2,thrC/yaaX,ECK120000989/ECK120002701,"(5021, 5233)"
3,yaaX/yaaA,ECK120002701/ECK120000009,"(5531, 5682)"
4,yaaA/yaaJ,ECK120000009/ECK120001508,"(6460, 6528)"


In [9]:
def get_feature_range(feat_id, feat_type):
    r = ()
    feat_id_col = 0
    range_col = "range"
    feat_id_col = "GENE_ID"
    feat_type_df = gene_df

    df = feat_type_df[feat_type_df[feat_id_col] == feat_id]
    if not df.empty:
        r = df.iloc[0][range_col]  # Assuming all rows describe the same range.
    l = [int(i) for i in r]
    return tuple(l)


# This will include all implicit intergenic regions, where get_genomic_features(...) does not.
# TODO: this may not handle wrapping around the genome from the last gene to the first gene. Write test for this.
def get_ordered_genetic_targets(genetic_targets):
    ordered_genetic_targets = sorted(
        genetic_targets, key=lambda k: k["range"][0])
    return ordered_genetic_targets


test_genetic_targets = [
    {'RegulonDB ID': "ECK120000003",
     "range": (2531463, 2532224)},
    {'RegulonDB ID': "ECK120000001",
     "range": (4265782, 4266861)},
    {'RegulonDB ID': "ECK120000002",
     "range": (795862, 796551)}]
assert(get_ordered_genetic_targets(test_genetic_targets) ==
       [{'RegulonDB ID': 'ECK120000002', 'range': (795862, 796551)},
        {'RegulonDB ID': 'ECK120000003', 'range': (2531463, 2532224)},
        {'RegulonDB ID': 'ECK120000001', 'range': (4265782, 4266861)}])


#Simply returning the first gene name in the second column that has the RegulonDB ID in the first column.
def get_gene_name(RegulonDB_ID, debug=False):
    if debug:
        display(RegulonDB_ID, gene_df[gene_df["GENE_ID"] == RegulonDB_ID])
    return gene_df[gene_df["GENE_ID"] == RegulonDB_ID].iloc[0]["GENE_NAME"]


def get_genetic_feat_JSON(mut_row):
    l = []
    for gene_id in mut_row["gene RegulonDB ID"]:
        l.append({
            # IDs are unique; assuming only be one entry per DF.
            "name": get_gene_name(gene_id),
            "RegulonDB ID": gene_id,
            "range": get_feature_range(gene_id, "gene RegulonDB ID"),
            "feature type": "gene"
        })

    intergenic_regions = get_feature_hit_set(mut_row["range"], intergenic_df, "range", "RegulonDB ID")
    for intergenic_region in intergenic_regions:
        l.append({
            "RegulonDB ID": intergenic_region,
            "name": str(get_gene_name(intergenic_region.split('/')[0])) + '/' + str(get_gene_name(intergenic_region.split('/')[1])),
            "range": intergenic_df[intergenic_df["RegulonDB ID"] == intergenic_region].iloc[0]["range"],
            "feature type": "unknown"
        })

    return l


all_muts_df["genetic features"] = all_muts_df.apply(lambda r: get_genetic_feat_JSON(r) if r.exp not in NON_K12_EXP_L else [], axis=1)
display(all_muts_df.head())

,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features
2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155..."
6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854..."
7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088..."
8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431..."
0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062..."


## E. coli B (REL606) genetic annotations

In [10]:
gene_synonym_df = pd.read_csv(
    "./data/RegulonDB10/object_synonym.txt",
    sep="\t",
    comment='#',
    header=None,
    quoting=3
)
gene_synonym_df.columns = ["OBJECT_ID", "OBJECT_SYNONYM_NAME", "OS_INTERNAL_COMMENT", "KEY_ID_ORG"]
gene_synonym_df.head()

,OBJECT_ID,OBJECT_SYNONYM_NAME,OS_INTERNAL_COMMENT,KEY_ID_ORG
0,ECK120000001,EG10001,NaN,ECK12
1,ECK120000001,ECK4045,NaN,ECK12
2,ECK120000001,b4053,NaN,ECK12
3,ECK120000001,alr5,NaN,ECK12
4,ECK120000002,b0764,NaN,ECK12


In [11]:
# TODO: replace into util.mut
def get_clean_mut_gene_list(gene_list_str):
    gene_list_str = gene_list_str.replace(" > ", " ")
    gene_list_str = gene_list_str.replace("]", "")
    gene_list_str = gene_list_str.replace("[", "")
    gene_list_str = gene_list_str.replace("<i><b>", '')
    gene_list_str = gene_list_str.replace("</b><BR>", '')
    gene_list_str = gene_list_str.replace("</i>", '')
    if "genes" in gene_list_str:
        start_idx = gene_list_str.rfind("genes")+len("genes")
        gene_list_str = gene_list_str[start_idx:]
    mut_gene_list = gene_list_str.split(", ")
    clean_mut_gene_list = [gene for gene in mut_gene_list]
    return clean_mut_gene_list


test_s = '<i><b>3 genes</b><BR>[manB], manC, [cpsG]</i>'
assert(get_clean_mut_gene_list(test_s) == ["manB", "manC", "cpsG"])


# TODO: put this within util.gene
def get_gene_RegDB_ID(gene_name):
    regDB_ID = ""
    df = gene_df[gene_df["GENE_NAME"] == gene_name]
    id_col_name = "GENE_ID"
    if len(df) == 0:  # Gene name used might be a synonym
        df = gene_synonym_df[gene_synonym_df["OBJECT_SYNONYM_NAME"] == gene_name]
        id_col_name = "OBJECT_ID"
    if len(df) > 0:
        regDB_ID = df.iloc[0][id_col_name]
    return regDB_ID


def get_genetic_feat_JSON_LTEE(mut_row):
    l = []
    mut_targ_annot = mut_row["mutation target annotation"]

    # Since trying to match genes to RegDB IDs, if genetic target is an "ECB" gene, will automatically be ignored.
    if '/' in mut_targ_annot and "," not in mut_targ_annot:
        regDB_ID_l = []
        interg_l = mut_targ_annot.split('/')
        for g in interg_l:
            regDB_ID = get_gene_RegDB_ID(g)
            if regDB_ID != "":
                regDB_ID_l.append(regDB_ID)
        if len(regDB_ID_l) == 2:  # only 2 gene annotations ever involved in intergenic region annotations.
            if len(gene_df[gene_df["GENE_ID"] == regDB_ID_l[0]]) and len(gene_df[gene_df["GENE_ID"] == regDB_ID_l[1]]):
                l.append({
                    "RegulonDB ID": regDB_ID_l[0] + '/' + regDB_ID_l[1],
                    "name": get_gene_name(regDB_ID_l[0]) + '/' + get_gene_name(regDB_ID_l[1]),
                    "feature type": "unknown"
                })
    # TODO: for refactoring, could combine this condition and the final condition into one
    elif ',' in mut_targ_annot:  # list of genes
        g_l = get_clean_mut_gene_list(mut_targ_annot)
        regDB_ID_s = set()  # making this a set so that no genes are duplicated.
        for g in g_l:
            regDB_ID = get_gene_RegDB_ID(g)
            if regDB_ID != "":
                regDB_ID_s.add(regDB_ID)
        if len(regDB_ID_s) > 0:
            for regDB_ID in regDB_ID_s:
                if len(gene_df[gene_df["GENE_ID"] == regDB_ID]) > 0:
                    l.append({
                        "RegulonDB ID": regDB_ID,
                        "name": get_gene_name(regDB_ID),
                        "feature type": "gene"
                    })
    else:  # just a single gene
        regDB_ID = get_gene_RegDB_ID(mut_targ_annot)
        if regDB_ID != "" and len(gene_df[gene_df["GENE_ID"] == regDB_ID]) > 0:
            l.append({
                "RegulonDB ID": regDB_ID,
                "name": get_gene_name(regDB_ID),
                "feature type": "gene"
            })

    return l


all_muts_df["genetic features"] = all_muts_df.apply(
    # will default to what is already in r["genetic features"] which is previously annotated above.
    lambda r: get_genetic_feat_JSON_LTEE(
        r) if r.exp in NON_K12_EXP_L else r["genetic features"],
    axis=1
)
all_muts_df.head()

,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features
2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155..."
6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854..."
7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088..."
8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431..."
0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062..."


In [12]:
all_muts_df.to_pickle("./data/1_5_df.pkl")